<a href="https://colab.research.google.com/github/jayshah1819/Anomaly-detection-in-payments/blob/main/Text_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I am using hugging face library

In [ ]:

!pip install diffusers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

In [1]:
#CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'torch' is not defined

In [ ]:
#CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:

# this will optimize GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

pipe.to(device)


In [ ]:


# Generated prompt
prompt = "two people are in a meeting in office"

#video frames
with torch.no_grad():
    video_frames = pipe(prompt, num_inference_steps=25, num_frames=20).frames

video_path = export_to_video(video_frames)

print("Video generated successfully:", video_path)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

It seems like you have activated model offloading by calling `enable_model_cpu_offload`, but are now manually moving the pipeline to GPU. It is strongly recommended against doing so as memory gains from offloading are likely to be lost. Offloading automatically takes care of moving the individual components vae, text_encoder, tokenizer, unet, scheduler to GPU when needed. To make sure offloading works as expected, you should consider moving the pipeline back to CPU: `pipeline.to('cpu')` or removing the move altogether if you use offloading.


  0%|          | 0/25 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 3)

In [ ]:
print(video_frames.shape)

(1, 20, 256, 256, 3)


In [ ]:
print("Original shape of video_frames:", video_frames.shape)

# Check if the first dimension of video_frames is 1 and remove it if necessary
if video_frames.shape[0] == 1:
    video_frames = video_frames.squeeze(0)

# Investigate the modified shape of video_frames
print("Modified shape of video_frames:", video_frames.shape)

# Modify the export_to_video() call
output_video_path = 'output_video.mp4'  # Specify the output video path
fps = 24  # Specify the frames per second for the output video

# Export the modified video frames to video
video_path = export_to_video(video_frames, output_video_path, fps=fps)

# Print the path to the generated video
print("Video generated successfully:", video_path)

Original shape of video_frames: (1, 20, 256, 256, 3)
Modified shape of video_frames: (20, 256, 256, 3)
Video generated successfully: output_video.mp4


In [ ]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /content


In [ ]:
from google.colab import files

# Download the video file
files.download('output_video.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>